In [1]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
import os

import logging
logging.basicConfig(level=logging.ERROR)
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent
from pydantic import BaseModel, Field
from google.adk.agents.parallel_agent import ParallelAgent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts


In [2]:
from yt_transcript import *

yt_url = "https://www.youtube.com/watch?v=sBuoMkJsMsA"

In [14]:
AGENT_MODEL = "gemini-2.0-flash"
root_agent = Agent(
    name="yt_agent_v0",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    ),
    output_key='yt_agent_v0', 
    tools=[get_youtube_transcript],
)

root_agent_1 = Agent(
    name="yt_agent_v1",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    ),
    output_key='yt_agent_v1',
    tools=[get_youtube_transcript],
)

root_agent_2 = Agent(
    name="yt_agent_v2",
    model=AGENT_MODEL, 
    description="Extracts transcripts from YouTube videos and summarizes the learnings related to a given topic.",
    instruction = (
    "You are an expert YouTube video summarizer. "
    "Given a topic and a YouTube video URL, use the 'get_youtube_transcript' tool to extract the transcript (returned as a JSON with key 'result'). "
    "From the transcript, generate a clear and concise summary that captures the key learnings relevant to the given topic. "
    "Your output should be only the final summarized content — answer in bullet points."
    ),
    output_key='yt_agent_v2',
    tools=[get_youtube_transcript],
)

In [15]:
session_service = InMemorySessionService()

APP_NAME = "yt_app"
USER_ID = "user_1"
SESSION_ID = "session_001" 

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
runner = Runner(
    agent=root_agent, 
    app_name=APP_NAME,   
    session_service=session_service #
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='yt_app', User='user_1', Session='session_001'
Runner created for agent 'yt_agent_v0'.


In [16]:
import asyncio
from google.genai import types 

async def call_agent_async(query: str):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." #default

  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break 

  print(f"<<< Agent Response: {final_response_text}")
  return final_response_text

In [17]:
text = await call_agent_async(f"how to swim? yt_url= {yt_url}")


>>> User Query: how to swim? yt_url= https://www.youtube.com/watch?v=sBuoMkJsMsA
<<< Agent Response: Here's a summary of how to swim freestyle, based on the YouTube video transcript:

*   **Step 1: Streamline:** Practice gliding on the surface with your hands forward, chin down, and blowing bubbles.
*   **Step 2: Arm Strokes:** Use a kickboard to practice pulling with your arms, keeping your elbows high and feeling the pressure on your hands and forearms. Then, try the same motion without the board, gliding with your hands on the surface and taking three strokes before breathing. To breathe, turn your head to the side as you pull with your hand, keeping your eye on the water.
*   **Step 3: Full Freestyle:** Instead of touching your hands together in front, glide with your hands one by one. Open your hands, relax your elbows, and reach as far as possible to touch the surface of the water. Stretch with your shoulders and nails, and avoid being stiff. Focus on smoothness and gliding on t

In [22]:
session = session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
session.state

{'yt_agent_v0': "Here's a summary of how to swim freestyle, based on the YouTube video transcript:\n\n*   **Step 1: Streamline:** Practice gliding on the surface with your hands forward, chin down, and blowing bubbles.\n*   **Step 2: Arm Strokes:** Use a kickboard to practice pulling with your arms, keeping your elbows high and feeling the pressure on your hands and forearms. Then, try the same motion without the board, gliding with your hands on the surface and taking three strokes before breathing. To breathe, turn your head to the side as you pull with your hand, keeping your eye on the water.\n*   **Step 3: Full Freestyle:** Instead of touching your hands together in front, glide with your hands one by one. Open your hands, relax your elbows, and reach as far as possible to touch the surface of the water. Stretch with your shoulders and nails, and avoid being stiff. Focus on smoothness and gliding on the surface."}

# Parallel Agent

In [9]:
parallel_research_agent = ParallelAgent(
    name="ParallelWebResearchAgent",
    sub_agents=[root_agent_1, root_agent_2]
)

In [ ]:
APP_NAME = "yt_app_advanced"
USER_ID = "user_2"
SESSION_ID = "session_002" 

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=parallel_research_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
    '''
    Helper function to call the agent with a query.
    '''
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

call_agent(f"how to swim? yt_url= {yt_url}")

Agent Response:  Here's a summary of the key learnings on how to swim freestyle from the provided transcript:

*   **Step 1: Streamline Position.** Practice blowing bubbles, stretching arms forward, and gliding on the surface with your head down and legs aligned.
*   **Step 2: Arm Strokes with a Board.** Use a kickboard to practice pulling through the water with your arms, keeping your elbows up. Focus on the pressure on your hands and forearms. Then, practice the same movement without the board, gliding with your hands on the surface.
*   **Step 3: Freestyle with Breathing.** Incorporate breathing into your stroke. Breathe after every third stroke by turning your head to the side as you pull with your hand on that side. Keep your eye on the water while breathing.
*   **Real Freestyle Technique:** Instead of touching your hands together, glide with your hands one by one. Open your hands like your shoulders, lift up your elbows, relax your hands, and try to reach as far as possible to t